In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

from src.create_data.create_data import create_data
from src.linear_regression.build_model import build_model
from src.ex_price_optimize.ex_optimize_price import optimize_price
from src.new_price_optimize.new_optimize_price import optimize_price_new,optimize_price_new_mosek
from src.new_price_optimize.new_optimize_price_bdf import optimize_price_new_bdf
from src.new_price_optimize.new_optimize_price_sdp import optimize_price_sdp_mosek
from src.evalut_price.evalut_optimal_prices import calculate_true_revenue

In [2]:
(((0.3/0.8)**2) * (0.8)**2)*20

1.7999999999999996

In [2]:
alpha, beta, X, Y = create_data(M=100, N=500, r_mean=0.8, r_std=0.1,delta=1.0)
intercept, coefficients = build_model(X, Y)

p_bar = np.full(100, 0.8)  # 平均価格ベクトルp̄をr_meanで初期化
epsilon = 0.09 * 10  # 許容誤差ε

In [4]:
epsilon

0.8999999999999999

In [4]:
beta

array([[-2.33671836,  2.75401395,  0.77488149, ...,  0.0294852 ,
         2.88432099,  2.41777032],
       [ 1.27245116, -2.24757706,  1.07746361, ...,  2.88145263,
         0.04078434,  1.3130269 ],
       [ 2.33673975,  0.47823623, -2.40955284, ...,  2.46220477,
         1.57343909,  2.45142366],
       ...,
       [ 1.76006765,  2.49481161,  2.0010841 , ..., -2.02178175,
         2.65188762,  2.21269709],
       [ 1.09498177,  2.88817217,  0.02786393, ...,  1.44108794,
        -2.4507995 ,  0.20011968],
       [ 1.21030217,  1.50354702,  2.19134749, ...,  0.41362972,
         0.70245985, -2.49034891]])

In [5]:
intercept

array([-1950.02052184,  1298.9998416 , -1357.21968633,  3111.52176885,
         -12.27859425,   510.91334368,  2842.56829908,  -572.18606329,
        1046.30268643,  -341.15479252,  -569.71914837,  1990.14035489,
         122.60050262,  2969.00553466,   125.11761624, -1223.79850386,
         549.85298902,  -645.02269753,   234.07284895,  -368.16381004,
        1004.9393009 ,   794.53659029,  -806.51639752,   613.91996466,
         695.0206582 ,  3383.56244636,   -38.9283112 ,  -607.69151097,
        1328.11084874,  -638.42014151,   487.93898265, -2420.10706167,
        1878.19624403,  -200.53318251,  -241.363635  ,  -735.25647922,
        1458.93607642,  1429.16575308, -1326.51516963,  -276.39802335,
         -76.54165297,  -418.92849944,   971.68638791, -1681.40487988,
       -1812.4839832 , -1965.15982211,   168.15259557,  -629.22058427,
          51.03105218,    51.92488312,   628.69940203,  1259.10688646,
        -477.93848019, -1049.92240597, -1376.99500815,  -994.02187641,
      

In [3]:
print(coefficients)

[[-270.12390224 -213.03499675    9.48200586 ...  301.51381893
   -55.84540141  -35.29315488]
 [  83.20560982  112.04346143 -154.22608583 ...  121.97932059
  -227.13400718  -85.30838557]
 [-118.71544426  -45.82889292  -38.46398872 ... -115.37774958
   -35.56633256   49.77056217]
 ...
 [-117.24060462    2.17535823 -116.31660706 ...    6.36342905
    29.96021056  -57.76914018]
 [  30.54053857   33.45536938  -36.43084832 ...  -55.07681819
   -27.79495436  -94.93660492]
 [-164.68973765  -47.86837219  -23.42707047 ...  -55.32259986
    91.38160222  288.4701321 ]]


In [6]:
M_I = np.zeros((5 + 1, 5 + 1))
M_I[0, 0] = 1

In [7]:
M_I

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [3]:
# sdpの解法を呼び出す
new_prices_sdp, new_max_revenue_sdp, new_solve_time = optimize_price_sdp_mosek(
    intercept,
    coefficients,
    p_bar=p_bar,
    epsilon=epsilon
)

--- SDP Dual Formulation with Price Recovery (MOSEK) ---
Optimal Prices: [0.9021 0.8526 0.8125 0.6582 0.9343 0.9607 0.7998 0.8537 0.7749 0.7613
 0.7595 0.911  0.7628 0.9231 0.7815 0.8794 0.8188 0.8717 0.9013 0.8286
 0.8732 0.8961 0.8405 0.9347 0.9747 0.8256 0.7772 0.9122 0.8335 0.895
 0.856  0.8709 0.7298 0.791  0.8204 0.7579 0.7659 0.9202 0.8308 0.9109
 0.6407 1.0099 0.8366 0.8171 0.8238 0.8766 0.85   0.823  0.8221 0.8115
 0.7956 0.7664 0.9205 0.9318 0.7299 0.936  0.8084 0.7913 0.7886 0.8261
 0.7886 1.0287 0.6206 0.9895 0.884  0.6767 0.8633 0.8944 0.9398 0.8318
 0.7624 0.8659 0.7655 0.8    0.8568 0.9707 0.9711 0.9294 0.9076 0.7902
 0.892  0.8567 0.8856 0.875  0.9112 0.8552 0.7484 0.8404 0.8799 0.7448
 1.0497 0.7905 0.6109 0.7493 0.8544 0.9316 0.9142 0.7925 0.7659 0.9733]
Maximum Revenue: 39014.0926
Solve Time: 0.030303 seconds


In [32]:
print(new_prices_sdp)

[0.90338502 1.21123579 0.95012446 0.92978528 0.86306351 0.99508295
 0.66341863 1.58275349 0.80534171 0.88626399]


In [33]:
new_prices_mosek, new_max_revenue_mosek, new_solve_time_mosek = optimize_price_new_mosek(
    intercept,
    coefficients,
    p_bar=p_bar,
    epsilon=epsilon
)

QCQP Optimization failed: The problem is not convex and does not follow DCP rules.


In [34]:

print(new_prices_mosek)

None


In [4]:
new_prices_mosek, new_max_revenue_mosek, new_solve_time_mosek = optimize_price_new(
    intercept,
    coefficients,
    p_bar=p_bar,
    epsilon=epsilon
)

Set parameter Username
Set parameter LicenseID to value 2659505
Academic license - for non-commercial use only - expires 2026-04-30
Set parameter TimeLimit to value 5
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  5

Optimize a model with 0 rows, 100 columns and 0 nonzeros
Model fingerprint: 0xd4d0aba1
Model has 5050 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [2e+00, 2e+00]
  Objective range  [8e+00, 3e+03]
  QObjective range [2e-01, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [6e+01, 6e+01]

Continuous model is non-convex -- solving as a MIP

Presolve time: 0.01s
Presolved: 10055 rows, 5152 columns, 22879 nonzeros
Presolved model has 49 quadrat

In [43]:
print(new_prices_sdp)

[0.81340932 0.81651345 0.72331623 0.78177799 0.91385185 0.87858026
 0.87313458 0.73719998 0.76584313 0.86389037 0.75781456 0.80333464
 0.74693736 0.78805124 0.7439493  0.73126166 0.71068996 0.75314534
 0.78542548 0.65300006 0.82232618 0.80643267 1.04007894 0.97714168
 0.79813015 0.81658199 0.77711363 0.83114716 1.03616695 0.86049663
 0.88865389 0.93739725 0.91615984 0.99782551 0.90503605 0.86373056
 0.80035488 0.78916699 1.05349068 0.92351823 0.88729179 0.6479312
 0.85946897 0.78534148 0.78696616 0.95075264 0.76825706 0.78893628
 0.72085583 0.78594557 0.87416423 0.96092448 0.94266122 0.79744133
 0.86304349 0.77529792 0.77951341 0.84216533 0.77075544 0.67149857
 0.83902834 0.85646536 0.87297608 0.85861359 0.89063593 0.5993991
 0.85146965 0.68992237 0.85971508 0.89180884 0.86533829 0.8890634
 0.77404197 1.0093534  0.83840086 0.71770032 0.89944197 0.71135263
 0.88521248 0.85054981 0.82331254 0.76508648 0.57759152 0.80483209
 0.92975969 0.70082487 0.82782138 0.88334305 0.88238339 0.8373883

In [44]:
print(new_prices_mosek)

[0.81340825 0.81651238 0.72332007 0.7817781  0.91384743 0.87857606
 0.87313143 0.7372019  0.76584386 0.86388763 0.75781554 0.80333386
 0.74693967 0.78805244 0.74395241 0.73126305 0.71069327 0.7531461
 0.78542573 0.65300596 0.82232406 0.80643209 1.04006832 0.97713435
 0.79812881 0.81658051 0.77711308 0.83114606 1.03615769 0.86049414
 0.88864824 0.93739119 0.91615375 0.99781699 0.90503127 0.86372802
 0.8003545  0.78916679 1.05348101 0.92351349 0.88728756 0.64793735
 0.85946655 0.78534235 0.78696627 0.95074677 0.76825772 0.78893605
 0.72085846 0.78594533 0.87416036 0.96091796 0.94265549 0.79744077
 0.86303987 0.7752991  0.77951431 0.84216094 0.7707557  0.67150321
 0.83902651 0.85646286 0.87297291 0.8586113  0.89063258 0.59940681
 0.85146692 0.68992589 0.85971321 0.89180398 0.86533479 0.88905918
 0.77404235 1.00934428 0.83839921 0.71770309 0.89943778 0.7113567
 0.88520783 0.85054804 0.8233122  0.76508893 0.57760077 0.80483119
 0.92975365 0.70082896 0.82781955 0.8833388  0.88237898 0.837386

In [5]:
print("SDP解法の結果:"
      f"\n新しい価格: {new_prices_sdp}\n"
      f"最大収益: {new_max_revenue_sdp}\n"
      f"解法時間: {new_solve_time:.4f}秒"
      )
print("Mosek解法の結果:"
      f"\n新しい価格: {new_prices_mosek}\n"
      f"最大収益: {new_max_revenue_mosek}\n"
      f"解法時間: {new_solve_time_mosek:.4f}秒"
      )

SDP解法の結果:
新しい価格: [0.90207523 0.85264233 0.81246174 0.65824327 0.93432665 0.96067623
 0.79977193 0.85367941 0.7748997  0.76131647 0.75945278 0.91103719
 0.76276368 0.92308339 0.78149167 0.87942187 0.81877914 0.87166748
 0.90125413 0.8285817  0.87315647 0.89611738 0.84052803 0.93468515
 0.97468344 0.82563342 0.77718827 0.9121869  0.83354502 0.89497465
 0.8559753  0.87092846 0.72984615 0.79103518 0.82037767 0.75785967
 0.76591048 0.92017228 0.83082143 0.91088709 0.64066598 1.00985259
 0.8365915  0.81711862 0.82380593 0.87662426 0.85001599 0.82300241
 0.82209426 0.81152151 0.79563431 0.76635512 0.92045943 0.93180198
 0.72987609 0.93604384 0.80835754 0.79134519 0.78859334 0.82606248
 0.78858491 1.02869925 0.62062042 0.98950408 0.8840246  0.67673359
 0.86326647 0.89443672 0.93977452 0.83180659 0.76236158 0.86586128
 0.76545316 0.79996121 0.8568465  0.97069953 0.97107355 0.92938855
 0.90761817 0.79020673 0.89200627 0.85668988 0.88556938 0.87497256
 0.91120595 0.85516079 0.74843179 0.84037081 

In [23]:
print(calculate_true_revenue(
    optimal_prices=new_prices_mosek,
    alpha_true=alpha,
    beta_true=beta
))

(2653.301223715102, array([22.12830895, 47.45573176, 68.35991654, 30.34054759, 34.09077841,
       53.21225924, 47.73738075, 38.75097085, 37.72126482, 41.60745354,
       47.21870101, 55.59436467, 69.91599668, 28.40357552, 43.60510852,
       35.02307507, 55.66032232, 54.00514617, 37.75822082, 26.41167269,
       51.05589068, 42.86171783, 37.39221722, 24.68779663, 72.28697508,
       24.40618   , 68.09440821, 52.13284075, 56.23577247, 66.43598647,
       30.08966716, 45.32982963, 72.2798618 , 26.44154935, 45.08510399,
       69.3708138 , 39.62376359, 23.89034984, 53.23401655, 21.1698377 ,
       58.06711283, 68.99356424, 61.95816954, 68.37324239, 61.99879469,
       72.06108465, 71.05945546, 71.84500009, 74.57511351, 75.11761556]))


In [24]:
print(calculate_true_revenue(
    optimal_prices=new_prices_sdp,
    alpha_true=alpha,
    beta_true=beta
))

(2645.7276999306546, array([17.77020789, 43.20840692, 70.2171685 , 34.53066093, 32.21846982,
       58.31294946, 41.9468614 , 37.88068992, 43.56472949, 40.71865311,
       45.89822652, 58.60398523, 76.81994672, 33.68929626, 40.46700998,
       26.71964066, 60.47240663, 61.52566913, 37.17941619, 34.1375021 ,
       56.81818753, 42.39454116, 38.77036959, 15.62663844, 72.97220111,
       18.0899454 , 64.37166224, 56.79446005, 57.13694009, 66.60153598,
       27.11569188, 44.0806663 , 73.78894581, 32.12039291, 48.8784667 ,
       74.7240275 , 41.3854962 , 25.41269454, 51.22326065, 24.78633215,
       58.97202062, 68.18206269, 59.18733905, 67.54285158, 61.12168694,
       69.77861466, 68.09847316, 69.99234552, 63.54290665, 69.91419166]))


In [25]:
print(new_solve_time)

0.011970043182373047


In [26]:
import numpy as np
from numpy.typing import NDArray

def evaluate_true_revenue(
    prices: NDArray,
    true_alpha: NDArray,
    true_beta: NDArray
) -> float:
    """
    与えられた価格ベクトルに基づき、真の需要関数 (alpha, beta) を用いて
    総売上を計算（評価）します。

    Args:
        prices (NDArray): 評価対象の価格ベクトル (p)。
        true_alpha (NDArray): 真の需要関数の切片ベクトル (α)。
        true_beta (NDArray): 真の需要関数の係数行列 (β)。

    Returns:
        float: 計算された総売上。
    """
    # 価格ベクトルが存在しない場合は0を返す
    if prices is None:
        return 0.0

    # 真の需要を計算: demand = alpha + beta * p
    # ここで beta は行列なので、行列とベクトルの積になります。
    true_demand = true_alpha + true_beta @ prices
    
    # 負の需要は0にクリップ（価格が高すぎて誰も買わないケースを想定）
    true_demand[true_demand < 0] = 0


    # 総売上を計算: revenue = p^T * demand
    total_revenue = prices @ true_demand
    
    return float(total_revenue)

In [27]:
print(evaluate_true_revenue(
    prices=new_prices_mosek,
    true_alpha=alpha,
    true_beta=beta
))

2653.301223715102


In [28]:
print(evaluate_true_revenue(new_prices_sdp,
    true_alpha=alpha,
    true_beta=beta
))

2645.7276999306546


In [29]:
p_bar = np.full(20, 0.8)  # 平均価格ベクトルp̄をr_meanで初期化
epsilon = 0.09 * np.linalg.norm(coefficients)  # 許容誤差ε
print("p_bar:", p_bar)
print("epsilon:", epsilon)


p_bar: [0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8
 0.8 0.8]
epsilon: 80.20293829747386


In [30]:
print(np.linalg.norm(coefficients))

891.1437588608208


In [31]:
len(p_bar)

20

In [20]:
import json
import numpy as np

def compare_prices_from_json(filename="experiment_results_delta60.json"):
    """
    実験結果のJSONファイルを読み込み、Gurobi(QCQP)とMOSEK(SDP)で得られた
    最適価格が一致するかどうかを確認し、Gurobiの計算時間を表示します。

    Args:
        filename (str): 入力するJSONファイル名。
    """
    # --- 1. JSONファイルの読み込み ---
    try:
        with open(filename, 'r') as f:
            results = json.load(f)
        print(f"✅ ファイル '{filename}' を正常に読み込みました。\n")
    except FileNotFoundError:
        print(f"❌ エラー: ファイル '{filename}' が見つかりません。")
        print("指定された実験スクリプトを先に実行して、JSONファイルを生成してください。")
        return
    except json.JSONDecodeError:
        print(f"❌ エラー: ファイル '{filename}' は有効なJSON形式ではありません。")
        return

    # --- 2. 各実験結果の比較 ---
    for experiment in results:
        exp_num = experiment.get("number", "N/A")
        gurobi_result = experiment.get("gurobi_qcqp_result", {})
        sdp_result = experiment.get("mosek_sdp_result", {})

        # --- 必要なデータの抽出 ---
        gurobi_prices = gurobi_result.get("optimal_prices")
        sdp_prices = sdp_result.get("optimal_prices")
        gurobi_runtime = gurobi_result.get("runtime")

        print(f"--- 実験番号: {exp_num} ---")

        # --- 価格リストの比較 ---
        # どちらかの価格リストが存在しない(None)場合は比較不可
        if gurobi_prices is None or sdp_prices is None:
            are_prices_close = False
            print("価格リストのどちらかが存在しないため、比較できません。")
        else:
            # np.allcloseで浮動小数点数の誤差を考慮して比較
            are_prices_close = np.allclose(gurobi_prices, sdp_prices)

        # --- 3. 結果の表示 ---
        print(f"Gurobi(QCQP)とMOSEK(SDP)の価格は一致するか？ -> {are_prices_close}")
        print(gurobi_prices)
        print(sdp_prices)
        print(f"Gurobi(QCQP)の計算時間: {gurobi_runtime:.6f} 秒")
        print("-" * 40)

compare_prices_from_json(filename="experiment_results_delta60.json")


✅ ファイル 'experiment_results_delta60.json' を正常に読み込みました。

--- 実験番号: 0 ---
Gurobi(QCQP)とMOSEK(SDP)の価格は一致するか？ -> True
[1.485354654546983, 0.9573920078516482, 1.1872868121479732, 0.9664373395314689, 1.1037171832264983, 0.9385660886116078, 0.627244251394589, 1.023202410626845, 0.8188055705477358, 1.09115723229776, 1.3014615289430536, 0.6018832316658046, 1.0174599723909301, 0.596880599478745, 1.031458207686347, 1.3598756167457346, 0.9381522783524731, 1.0133654138654404, 0.9968632109435791, 0.8439285872840891]
[1.4853468212573853, 0.9573909469216895, 1.1872824530011281, 0.9664375875490585, 1.1037113279429076, 0.9385637075017041, 0.6272447548027308, 1.0231976588883025, 0.8188033817149212, 1.0911513788095837, 1.3014540875630791, 0.6018871374226686, 1.0174566795890818, 0.5968813678830165, 1.0314536665314227, 1.359867819313945, 0.9381505596972884, 1.013360764162507, 0.9968615095942625, 0.8439282398805104]
Gurobi(QCQP)の計算時間: 60.001736 秒
----------------------------------------
--- 実験番号: 1 ---
Gurobi